In [19]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline
%matplotlib qt

In [20]:
Bank_America = pd.read_csv("Bank_of_America_data.csv")
#german_Bank = pd.read_table("german.data",sep=" ")
#taiwan_Bank = pd.read_excel("default of credit card clients.xls",header=1,index_col=0)

### I/ American Bank

In [21]:
Bank_America_1=Bank_America.dropna(thresh=6)    #On va supprimer les lignes qui contiennent moins de 6 valeurs non manquantes


values = {'JOB': 'Other'}
Bank_America_1=Bank_America_1.fillna(value=values)    
#On va remplacer les valeurs manquantes par le job Other

Bank_America_1['MORTDUE'] = Bank_America_1['MORTDUE'].fillna(value=Bank_America_1['VALUE']*0.8) 
#On va remplacer les valeurs manquantes de la colonne MORTDUE par 0.8*la valeur de la 
#colonne VALUE du meme lgne (parcequ'on a remarqué que c'est le cas pour les autres valeurs
#présentent) 
Bank_America_1['VALUE'] = Bank_America_1['VALUE'].fillna(value=Bank_America_1['MORTDUE']*1.2) 
#On va  remplacer les valeurs manquantes de la colonne VALUE par 1.2*la valeur de la 
#colonne MORTDUE du meme ligne (parcequ'on a remarqué que c'est le cas pour les autres 
#valeurs présentent)

#On va supprimer les 15 lignes que nous avons pas pu remplacer leurs colonne 
#MORTDUE et VALUE puisque ces deux colonnes n'ont ni une valeur dans MORTDUE ni dans VALUE
Bank_America_1.dropna(subset=["MORTDUE"],inplace=True)
Bank_America_1.dropna(subset=["VALUE"],inplace=True)

# On va faire un traitement qui nous permettera de remplir les valeurs manquantes REASON 
#et ceci en cherchant la majorité des clients ayant un profil similaire au profil de 
#ce client et ensuite on affecte la valeur de REASON de la majorité à ce champ vide 
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["REASON"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        d=Bank_America_1.loc[(Bank_America_1["LOAN"]<x*1.2)&(x*0.8<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.2)&(y*0.8<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.2)&(z*0.8<Bank_America_1["VALUE"])]
        d1=d.loc[(d["REASON"]=="HomeImp")]
        s1=d1.shape[0]
        d2=d.loc[(d["REASON"]=="DebtCon")]
        s2=d2.shape[0]
        if(s1>s2):
            Bank_America_1["REASON"][index]=d1.iloc[0]["REASON"]
        elif (s2>s1):
            Bank_America_1["REASON"][index]=d2.iloc[0]["REASON"]

# Une seule ligne n'a pas eu de REASON parcequ'il y a pas un profil proche de ce client donc on va supprimer cette ligne 
Bank_America_1.dropna(subset=["REASON"],inplace=True)

# On va faire un traitement qui nous permettera de remplir les valeurs manquantes de YOJ et ceci en cherchant la majorité des clients ayant un profil similaire au profil de ce client et on calcule leurs mediane ensuite on affecte la valeur de la mediane à ce champ vide 
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["YOJ"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["YOJ"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.2)&(x*0.8<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.2)&(y*0.8<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.2)&(z*0.8<Bank_America_1["VALUE"])]
        l=np.median(d["YOJ"])
        if np.isnan(l):
            l=2.0
        Bank_America_1["YOJ"][index]=round(l, 1)
        
# Meme principe que la précédente
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["DEROG"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["DEROG"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=d["DEROG"].iloc[0]
        else:
            l=0.0 
        Bank_America_1["DEROG"][index]=round(l, 1)
        
# Meme principe que la précédente       
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["DELINQ"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        w=Bank_America_1["YOJ"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["DELINQ"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=d["DELINQ"].iloc[0]
        else:
            l=0.0 
        Bank_America_1["DELINQ"][index]=round(l, 1)
        
# Meme principe que la précédente 
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["CLAGE"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        w=Bank_America_1["YOJ"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["CLAGE"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=np.median(d["CLAGE"])
        else:
            l=173.466667 
        Bank_America_1["CLAGE"][index]=round(l, 6)

# Meme principe que la précédente        
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["CLNO"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["CLNO"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=np.median(d["CLNO"])
        else:
            l=20.0
        Bank_America_1["CLNO"][index]=round(l, 1)

# Meme principe que la précédente                
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["NINQ"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["NINQ"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=np.median(d["NINQ"])
        else:
            l=1.0
        Bank_America_1["NINQ"][index]=round(l, 1)
        
# Meme principe que la précédente
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["DEBTINC"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["DEBTINC"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=np.median(d["DEBTINC"])
        else:
            l=34.898413
        Bank_America_1["DEBTINC"][index]=round(l, 6)
        
# one hot encoding       
df1 = pd.get_dummies(Bank_America_1.REASON)
df2 = pd.get_dummies(Bank_America_1.JOB)
Bank_America_After_Encoding=pd.concat([Bank_America_1,df1,df2],axis=1)
Bank_America_After_Encoding.drop(labels=['REASON','JOB'], axis =1, inplace = True)


# Mettre la column cible dans un vecteur à part et la supprimer de la dataframe
cible_America=Bank_America_After_Encoding["BAD"]
Bank_America_Final=Bank_America_After_Encoding.drop(labels=['BAD'], axis =1)

# DataFrame surlaquelle on va appliquer nos algorithmes:
Bank_America_Final

C:\Users\iheb_\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\iheb_\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\iheb_\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\iheb_\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,DebtCon,HomeImp,Mgr,Office,Other,ProfExe,Sales,Self
0,1100,25860.0,39025.0,10.5,0.0,0.0,94.366667,1.0,9.0,34.898413,0,1,0,0,1,0,0,0
1,1300,70053.0,68400.0,7.0,0.0,2.0,121.833333,0.0,14.0,34.898413,0,1,0,0,1,0,0,0
2,1500,13500.0,16700.0,4.0,0.0,0.0,149.466667,1.0,10.0,34.898413,0,1,0,0,1,0,0,0
4,1700,97800.0,112000.0,3.0,0.0,0.0,93.333333,0.0,14.0,30.635165,0,1,0,1,0,0,0,0
5,1700,30548.0,40320.0,9.0,0.0,0.0,101.466002,1.0,8.0,37.113614,0,1,0,0,1,0,0,0
6,1800,48649.0,57037.0,5.0,3.0,2.0,77.100000,1.0,17.0,35.891654,0,1,0,0,1,0,0,0
7,1800,28502.0,43034.0,11.0,0.0,0.0,88.766030,0.0,8.0,36.884894,0,1,0,0,1,0,0,0
8,2000,32700.0,46740.0,3.0,0.0,2.0,216.933333,1.0,12.0,36.388274,0,1,0,0,1,0,0,0
9,2000,49800.0,62250.0,16.0,0.0,0.0,115.800000,0.0,13.0,34.898413,0,1,0,0,0,0,1,0
11,2000,20627.0,29800.0,11.0,0.0,1.0,122.533333,1.0,9.0,34.898413,0,1,0,1,0,0,0,0


In [22]:
Bank_America = pd.read_csv("Bank_of_America_data.csv")
Bank_America_1=Bank_America.dropna(thresh=6)    #On va supprimer les lignes qui contiennent moins de 6 valeurs non manquantes


values = {'JOB': 'Other'}
Bank_America_1=Bank_America_1.fillna(value=values)    
#On va remplacer les valeurs manquantes par le job Other

Bank_America_1['MORTDUE'] = Bank_America_1['MORTDUE'].fillna(value=Bank_America_1['VALUE']*0.8) 
#On va remplacer les valeurs manquantes de la colonne MORTDUE par 0.8*la valeur de la 
#colonne VALUE du meme lgne (parcequ'on a remarqué que c'est le cas pour les autres valeurs
#présentent) 
Bank_America_1['VALUE'] = Bank_America_1['VALUE'].fillna(value=Bank_America_1['MORTDUE']*1.2) 
#On va  remplacer les valeurs manquantes de la colonne VALUE par 1.2*la valeur de la 
#colonne MORTDUE du meme ligne (parcequ'on a remarqué que c'est le cas pour les autres 
#valeurs présentent)

#On va supprimer les 15 lignes que nous avons pas pu remplacer leurs colonne 
#MORTDUE et VALUE puisque ces deux colonnes n'ont ni une valeur dans MORTDUE ni dans VALUE
Bank_America_1.dropna(subset=["MORTDUE"],inplace=True)
Bank_America_1.dropna(subset=["VALUE"],inplace=True)

# On va faire un traitement qui nous permettera de remplir les valeurs manquantes REASON 
#et ceci en cherchant la majorité des clients ayant un profil similaire au profil de 
#ce client et ensuite on affecte la valeur de REASON de la majorité à ce champ vide 
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["REASON"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        d=Bank_America_1.loc[(Bank_America_1["LOAN"]<x*1.2)&(x*0.8<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.2)&(y*0.8<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.2)&(z*0.8<Bank_America_1["VALUE"])]
        d1=d.loc[(d["REASON"]=="HomeImp")]
        s1=d1.shape[0]
        d2=d.loc[(d["REASON"]=="DebtCon")]
        s2=d2.shape[0]
        if(s1>s2):
            Bank_America_1["REASON"][index]=d1.iloc[0]["REASON"]
        elif (s2>s1):
            Bank_America_1["REASON"][index]=d2.iloc[0]["REASON"]

# Une seule ligne n'a pas eu de REASON parcequ'il y a pas un profil proche de ce client donc on va supprimer cette ligne 
Bank_America_1.dropna(subset=["REASON"],inplace=True)

# On va faire un traitement qui nous permettera de remplir les valeurs manquantes de YOJ et ceci en cherchant la majorité des clients ayant un profil similaire au profil de ce client et on calcule leurs mediane ensuite on affecte la valeur de la mediane à ce champ vide 
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["YOJ"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["YOJ"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.2)&(x*0.8<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.2)&(y*0.8<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.2)&(z*0.8<Bank_America_1["VALUE"])]
        l=np.median(d["YOJ"])
        if np.isnan(l):
            l=2.0
        Bank_America_1["YOJ"][index]=round(l, 1)
        
# Meme principe que la précédente
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["DEROG"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["DEROG"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=d["DEROG"].iloc[0]
        else:
            l=0.0 
        Bank_America_1["DEROG"][index]=round(l, 1)
        
# Meme principe que la précédente       
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["DELINQ"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        w=Bank_America_1["YOJ"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["DELINQ"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=d["DELINQ"].iloc[0]
        else:
            l=0.0 
        Bank_America_1["DELINQ"][index]=round(l, 1)
        
# Meme principe que la précédente 
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["CLAGE"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        w=Bank_America_1["YOJ"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["CLAGE"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=np.median(d["CLAGE"])
        else:
            l=173.466667 
        Bank_America_1["CLAGE"][index]=round(l, 6)

# Meme principe que la précédente        
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["CLNO"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["CLNO"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=np.median(d["CLNO"])
        else:
            l=20.0
        Bank_America_1["CLNO"][index]=round(l, 1)

# Meme principe que la précédente                
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["NINQ"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["NINQ"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=np.median(d["NINQ"])
        else:
            l=1.0
        Bank_America_1["NINQ"][index]=round(l, 1)
        
# Meme principe que la précédente
for index,row in Bank_America_1.iterrows():
    if (pd.isnull(row["DEBTINC"])):
        x=Bank_America_1["LOAN"][index]
        y=Bank_America_1["MORTDUE"][index]
        z=Bank_America_1["VALUE"][index]
        t=Bank_America_1["REASON"][index]
        v=Bank_America_1["JOB"][index]
        d=Bank_America_1.loc[(pd.notna(Bank_America_1["DEBTINC"]))&(Bank_America_1["JOB"]==v)&(Bank_America_1["REASON"]==t)&(Bank_America_1["LOAN"]<x*1.5)&(x*0.5<Bank_America_1["LOAN"])&(Bank_America_1["MORTDUE"]<y*1.5)&(y*0.5<Bank_America_1["MORTDUE"])&(Bank_America_1["VALUE"]<z*1.5)&(z*0.5<Bank_America_1["VALUE"])]
        if len(d)!=0:
            l=np.median(d["DEBTINC"])
        else:
            l=34.898413
        Bank_America_1["DEBTINC"][index]=round(l, 6)
        
# one hot encoding       
df1 = pd.get_dummies(Bank_America_1.REASON)
df2 = pd.get_dummies(Bank_America_1.JOB)
Bank_America_After_Encoding=pd.concat([Bank_America_1,df1,df2],axis=1)
Bank_America_After_Encoding.drop(labels=['REASON','JOB'], axis =1, inplace = True)
Bank_America_After_Encoding

C:\Users\iheb_\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\iheb_\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\iheb_\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\iheb_\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,BAD,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,DebtCon,HomeImp,Mgr,Office,Other,ProfExe,Sales,Self
0,1,1100,25860.0,39025.0,10.5,0.0,0.0,94.366667,1.0,9.0,34.898413,0,1,0,0,1,0,0,0
1,1,1300,70053.0,68400.0,7.0,0.0,2.0,121.833333,0.0,14.0,34.898413,0,1,0,0,1,0,0,0
2,1,1500,13500.0,16700.0,4.0,0.0,0.0,149.466667,1.0,10.0,34.898413,0,1,0,0,1,0,0,0
4,0,1700,97800.0,112000.0,3.0,0.0,0.0,93.333333,0.0,14.0,30.635165,0,1,0,1,0,0,0,0
5,1,1700,30548.0,40320.0,9.0,0.0,0.0,101.466002,1.0,8.0,37.113614,0,1,0,0,1,0,0,0
6,1,1800,48649.0,57037.0,5.0,3.0,2.0,77.100000,1.0,17.0,35.891654,0,1,0,0,1,0,0,0
7,1,1800,28502.0,43034.0,11.0,0.0,0.0,88.766030,0.0,8.0,36.884894,0,1,0,0,1,0,0,0
8,1,2000,32700.0,46740.0,3.0,0.0,2.0,216.933333,1.0,12.0,36.388274,0,1,0,0,1,0,0,0
9,1,2000,49800.0,62250.0,16.0,0.0,0.0,115.800000,0.0,13.0,34.898413,0,1,0,0,0,0,1,0
11,1,2000,20627.0,29800.0,11.0,0.0,1.0,122.533333,1.0,9.0,34.898413,0,1,0,1,0,0,0,0


In [23]:
Bank_America_Final.head()

,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,DebtCon,HomeImp,Mgr,Office,Other,ProfExe,Sales,Self
0,1100,25860.0,39025.0,10.5,0.0,0.0,94.366667,1.0,9.0,34.898413,0,1,0,0,1,0,0,0
1,1300,70053.0,68400.0,7.0,0.0,2.0,121.833333,0.0,14.0,34.898413,0,1,0,0,1,0,0,0
2,1500,13500.0,16700.0,4.0,0.0,0.0,149.466667,1.0,10.0,34.898413,0,1,0,0,1,0,0,0
4,1700,97800.0,112000.0,3.0,0.0,0.0,93.333333,0.0,14.0,30.635165,0,1,0,1,0,0,0,0
5,1700,30548.0,40320.0,9.0,0.0,0.0,101.466002,1.0,8.0,37.113614,0,1,0,0,1,0,0,0


In [24]:
from sklearn.model_selection import train_test_split

all_X = Bank_America_After_Encoding.iloc[:,:18]
all_y = Bank_America_After_Encoding['DebtCon']

train_X, test_X, train_y, test_y = train_test_split(
    all_X, all_y, test_size=0.2,random_state=0)

In [25]:
all_X.head()


,BAD,LOAN,MORTDUE,VALUE,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC,DebtCon,HomeImp,Mgr,Office,Other,ProfExe,Sales
0,1,1100,25860.0,39025.0,10.5,0.0,0.0,94.366667,1.0,9.0,34.898413,0,1,0,0,1,0,0
1,1,1300,70053.0,68400.0,7.0,0.0,2.0,121.833333,0.0,14.0,34.898413,0,1,0,0,1,0,0
2,1,1500,13500.0,16700.0,4.0,0.0,0.0,149.466667,1.0,10.0,34.898413,0,1,0,0,1,0,0
4,0,1700,97800.0,112000.0,3.0,0.0,0.0,93.333333,0.0,14.0,30.635165,0,1,0,1,0,0,0
5,1,1700,30548.0,40320.0,9.0,0.0,0.0,101.466002,1.0,8.0,37.113614,0,1,0,0,1,0,0


### Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
from sklearn.model_selection import cross_val_score
import numpy as np

lr = LogisticRegression()
scores = cross_val_score(lr, all_X, all_y, cv=18)
np.mean(scores)

C:\Users\iheb_\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\iheb_\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\iheb_\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\iheb_\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\iheb_\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

0.7243129798912575

In [27]:
lr.fit(train_X, train_y)
predictions_lr = lr.predict(test_X)

C:\Users\iheb_\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [28]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score


#metrics.precision_score
#metrics.recall_score

#metrics.roc_auc_score

#metrics.mean_squared_error

f1 = f1_score(test_y, predictions_lr)
accuracy = accuracy_score(test_y, predictions_lr)
recall = recall_score(test_y, predictions_lr)
precision = precision_score(test_y, predictions_lr)
print([accuracy, recall, precision, f1])

[0.7254063301967494, 0.9829683698296837, 0.7246636771300449, 0.8342798141455859]


In [29]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(test_y, predictions_lr)
pd.DataFrame(conf_matrix, columns=['0', '1'], index=[['0', '1']])

,0,1
0,40,307
1,14,808


### SVM

In [30]:
from sklearn.svm import SVC, LinearSVC
svc = SVC(probability=True)
svc.fit(train_X, train_y)
predictions_svm = svc.predict(test_X)
acc_svc = accuracy_score(test_y, predictions_svm)
acc_svc

C:\Users\iheb_\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.7031650983746792

### KNN

In [31]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 4)
knn.fit(train_X, train_y)
predictions_knn = knn.predict(test_X)
acc_knn = accuracy_score(test_y, predictions_knn)
acc_knn

0.7827202737382378

### Naive Bayes	

In [32]:
from sklearn.naive_bayes import GaussianNB

gaussian = GaussianNB()
gaussian.fit(train_X, train_y)
predictions = gaussian.predict(test_X)
acc_gaussian = accuracy_score(test_y, predictions)
acc_gaussian

0.7596236099230111

### Decision Tree

In [33]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier()
decision_tree.fit(train_X, train_y)
predictions_dt = decision_tree.predict(test_X)
acc_decision_tree = accuracy_score(test_y, predictions_dt)
acc_decision_tree

1.0

## Kmeans

In [34]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [35]:
#Déterminer le nombre de clusters "Elbow"
L = []
for i in range(1,18):
    model = KMeans(n_clusters=i)
    model.fit(Bank_America_Final)
    L.append(model.inertia_)
plt.plot(range(1,18),L);

In [36]:
from sklearn.cluster import KMeans 
kmeans = KMeans(n_clusters=2, precompute_distances='auto')
kmeans.fit(Bank_America_Final)
#print(kmeans.cluster_centers_)
y_kmeans = kmeans.fit_predict(Bank_America_Final)
plt.scatter(Bank_America_Final.values[y_kmeans == 0, 0], Bank_America_Final.values[y_kmeans == 0, 1], s = 100, c = 'blue', label = 'Cluster 1')
plt.scatter(Bank_America_Final.values[y_kmeans == 1, 0], Bank_America_Final.values[y_kmeans == 1, 1], s = 100, c = 'red', label = 'Cluster 2')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 200, c = 'green', label = 'Centroids')
plt.legend()
plt.show()


In [37]:
# Evaluer les résultats de k-Means 
#index triés des groupes 
idk = np.argsort(kmeans.labels_)
#affichage des observations et leurs groupes 
print(pd.DataFrame(Bank_America_Final.index[idk],kmeans.labels_[idk]));

       0
0   5057
0   4926
0   4927
0   4928
0   4929
0   3688
0   4931
0   4932
0   4935
0   4937
0   3684
0   1122
0   2506
0   4943
0   1118
0   4944
0   3679
0   1114
0   3689
0   4949
0   3691
0   4919
0   4888
0   4891
0   3712
0   4894
0   4900
0   1167
0   1166
0   4902
..   ...
1   2062
1   2060
1   2059
1   2058
1   2056
1   2053
1   2066
1   2101
1   2078
1   2080
1   2099
1   2098
1   2097
1   2096
1   2095
1   2094
1   2093
1   2092
1   2079
1   2091
1   2089
1   2088
1   2086
1   2085
1   2084
1   2083
1   2082
1   2081
1   2090
1   5959

[5843 rows x 1 columns]


In [38]:
kmeans.labels_
V_Labels=Bank_America_Final.iloc[:,6]
pd.crosstab(V_Labels,kmeans.labels_)

col_0,0,1
CLAGE,,
0.000000,1,1
0.486711,0,1
0.507115,1,0
2.033333,0,1
2.820786,1,0
3.044384,1,0
4.412770,1,0
5.243341,0,1
6.133333,1,0


In [39]:
#Enregistrer le DataFrame Final dans un fichier CSV
Bank_America_Final.to_csv(r'C:\Users\iheb_\OneDrive\Desktop\Bank_America_Final.txt')

### II/ taiwanese Bank

In [40]:
# Pour la banque de Taiwan il y a pas de données manquantes Donc on va faire directement l'encodage (One Hot Encoder)
taiwan_Bank_1=taiwan_Bank
# Il ya des valeurs autres que 1,2 et 3 alors on a remplacer ces valeurs par "Others" qui vaut 3
taiwan_Bank_1.MARRIAGE.replace([0], [3], inplace=True)
# Il ya des valeurs autres que 1,2,3 et 4 dans la colonne EDUCATION alors on a remplacer ces valeurs par "Other" qui vaut 4
taiwan_Bank_1.EDUCATION.replace([0], [4], inplace=True)
taiwan_Bank_1.EDUCATION.replace([5], [4], inplace=True)
taiwan_Bank_1.EDUCATION.replace([6], [4], inplace=True)
# one hot encoding
df1 = pd.get_dummies(taiwan_Bank_1.SEX)
df1.rename(columns = {1:'Male',2:'Female'}, inplace = True)
df2 = pd.get_dummies(taiwan_Bank_1.MARRIAGE)
df2.rename(columns = {1:'Mar_Married',2:'Mar_Single',3:'Mar_Others'}, inplace = True)
df3 = pd.get_dummies(taiwan_Bank_1.EDUCATION)
df3.rename(columns = {1:'Ed_GS',2:'Ed_HS',3:'Ed_Univ',4:'Ed_Other'}, inplace = True)
taiwan_Bank_1 = pd.concat([taiwan_Bank_1,df1,df2,df3], axis = 1)
taiwan_Bank_1.drop(labels=['SEX','MARRIAGE','EDUCATION'], axis =1, inplace = True)

# Mettre la column cible dans un vecteur à part et la supprimer de la dataframe
cible_taiwan=taiwan_Bank_1["default payment next month"]
taiwan_Bank_Final=taiwan_Bank_1.drop(labels=['default payment next month'], axis =1)

# DataFrame surlaquelle on va appliquer nos algorithmes:
taiwan_Bank_Final

NameError: name 'taiwan_Bank' is not defined

In [ ]:
# Enregistrer le DataFrame Final dans un fichier CSV
taiwan_Bank_Final.to_csv(r'C:\Users\iheb_\OneDrive\Desktop\Python_Projet_Final\taiwan_Bank_Final.txt')

### III/ German Bank

In [ ]:
# Pour la banque de Taiwan il y a pas de données manquantes Donc on va faire directement l'encodage (One Hot Encoder)
german_Bank_1=german_Bank
# one hot encoding
df1 = pd.get_dummies(german_Bank_1.Status)
df1.rename(columns = {'A11':'Status_A11','A12':'Status_A12','A13':'Status_A13','A14':'Status_A14'}, inplace = True)
df2 = pd.get_dummies(german_Bank_1.credit_history)
df2.rename(columns = {'A30':'CredHist_A30','A31':'CredHist_A31','A32':'CredHist_A32','A33':'CredHist_A33','A34':'CredHist_A34'}, inplace = True)
df3 = pd.get_dummies(german_Bank_1.Purpose)
df3.rename(columns = {'A40':'Purpose_A40','A41':'Purpose_A41','A42':'Purpose_A42','A43':'Purpose_A43','A44':'Purpose_A44','A45':'Purpose_A45','A46':'Purpose_A46','A47':'Purpose_A47','A48':'Purpose_A48','A49':'Purpose_A49','A410':'Purpose_A410'}, inplace = True)
df4 = pd.get_dummies(german_Bank_1.Saving_amount)
df4.rename(columns = {'A61':'SaveAccount_A61','A62':'SaveAccount_A62','A63':'SaveAccount_A63','A64':'SaveAccount_A64','A65':'SaveAccount_A65'}, inplace = True)
df5 = pd.get_dummies(german_Bank_1.YOJ)
df5.rename(columns = {'A71':'YOJ_A71','A72':'YOJ_A72','A73':'YOJ_A73','A74':'YOJ_A74','A75':'YOJ_A75'}, inplace = True)
df6 = pd.get_dummies(german_Bank_1.Status_SEX)
df6.rename(columns = {'A91':'Status&Sex_A91','A92':'Status&Sex_A92','A93':'Status&Sex_A93','A94':'Status&Sex_A94','A95':'Status&Sex_A95'}, inplace = True)
df7 = pd.get_dummies(german_Bank_1.debtors_guarantors)
df7.rename(columns = {'A101':'DebGuarant_A101','A102':'DebGuarant_A102','A103':'DebGuarant_A103'}, inplace = True)
df9 = pd.get_dummies(german_Bank_1.Property)
df9.rename(columns = {'A121':'Property_A121','A122':'Property_A122','A123':'Property_A123','A124':'Property_A124'}, inplace = True)
df10 = pd.get_dummies(german_Bank_1.installment_plans)
df10.rename(columns = {'A141':'InstallPlan_A141','A142':'InstallPlan_A142','A143':'InstallPlan_A143'}, inplace = True)
df11 = pd.get_dummies(german_Bank_1.Housing)
df11.rename(columns = {'A151':'Housing_A151','A152':'Housing_A152','A153':'Housing_A153'}, inplace = True)
df12 = pd.get_dummies(german_Bank_1.Job)
df12.rename(columns = {'A171':'Job_A171','A172':'Job_A172','A173':'Job_A173','A174':'Job_A174'}, inplace = True)
df13 = pd.get_dummies(german_Bank_1.Telephone)
df13.rename(columns = {'A191':'Tel_None','A192':'Tel_Registred'}, inplace = True)
df14 = pd.get_dummies(german_Bank_1.Foreign_warker)
df14.rename(columns = {'A201':'ForeignWorker_Yes','A202':'ForeignWorker_No'}, inplace = True)

german_Bank_1 = pd.concat([german_Bank_1,df1,df2,df3,df4,df5,df6,df7,df9,df10,df11,df12,df13,df14], axis = 1)
german_Bank_1.drop(labels=['Status','credit_history','Purpose','Saving_amount','YOJ','Status_SEX','debtors_guarantors','Property','installment_plans','Housing','Job','Telephone','Foreign_warker'], axis =1, inplace = True)

# Mettre la column cible dans un vecteur à part et la supprimer de la dataframe
cible_german=german_Bank_1["Cost"]
german_Bank_Final=german_Bank_1.drop(labels=['Cost'], axis =1)

# DataFrame surlaquelle on va appliquer nos algorithmes:
german_Bank_Final


In [ ]:
# Enregistrer le DataFrame Final dans un fichier CSV
german_Bank_Final.to_csv(r'C:\Users\iheb_\OneDrive\Desktop\Python_Projet_Final\german_Bank_Final.txt')

## -Kmeans

In [ ]:
import numpy as np
from  sklearn.cluster import KMeans
km= KMeans(n_clusters=2).fit(Bank_America_Final)
Bank_America_Final['CLUSTER']=km.labels_
table=pd.crosstab(Bank_America_Final['BAD'],Bank_America_Final['CLUSTER'])
table  #VP  VN
       #FN  FP

In [ ]:
Bank_America_Final.head()